## IF702 - REDES NEURAIS

**Grupo:**
- Adriano Felipe Cabral Filho (afcf)
- Felipe Rodrigues de Souza (frs3)
- Lucas Vinícius José da Silva (lvjs)
- Pedro José Carneiro de Souza (pjcs) 
- Vitor Sousa Silva (vss2)


# Reconhecimento de Dígitos com CNN

##Importação de datasets e tratamento dos dados

In [ ]:
!pip install keras-tuner --quiet
!pip install optuna --quiet
!pip install ipython-autotime --quiet
from keras_tuner import RandomSearch
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import optuna

#Loading Data
(train_images,train_labels),(test_images,test_labels)=tf.keras.datasets.mnist.load_data()

#Reshape data
train_images=train_images.reshape((60000,28,28,1))
test_images=test_images.reshape((10000,28,28,1))

#Normalize
train_images,test_images=train_images/255.0 ,test_images/255.0

## Construindo a CNN

In [ ]:
#Building a CNN Model
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3),activation='relu',input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [ ]:
model.fit(train_images,train_labels,epochs=1,batch_size=10)
scores = model.evaluate(test_images,test_labels, verbose=0)

6000/6000 [==============================] - 16s 2ms/step - loss: 0.1172 - accuracy: 0.9637


In [ ]:
def build_model(hp):
    # create model object
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3,3),activation='relu',input_shape=(28,28,1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64, (3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.summary()
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
    return model

In [ ]:
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5, directory='./untitled_project/trial/')
# search best parameter
tuner.search(train_images,train_labels,epochs=3,validation_data=(train_images,train_labels))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/trial/untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_____________

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [ ]:
# Sugestão de Adriano (128 layers na 3a CNN)
def build_model2(hp):
    # create model object
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3,3),activation='relu',input_shape=(28,28,1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64, (3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.summary()
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
    return model

tuner2 = RandomSearch(build_model2,
                    objective='val_accuracy',
                    max_trials = 5,directory='./untitled_project/trial2/')

tuner2.search(train_images,train_labels,epochs=3,validation_data=(train_images,train_labels))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/trial2/untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
____________

In [ ]:
# Sugestão de Adriano (128 layers na 3a CNN com dilation_rate menor=2)
def build_model3(hp):
    # create model object
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (2,2),activation='relu',input_shape=(28,28,1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64, (2,2),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(128,(2,2),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.summary()
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
    return model

tuner3 = RandomSearch(build_model3,
                    objective='val_accuracy',
                    max_trials = 5, directory='./untitled_project/trial3/')

tuner3.search(train_images,train_labels,epochs=3,validation_data=(train_images,train_labels))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/trial3/untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 32)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         32896     
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
____________

In [ ]:
from keras.layers import LSTM
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix, classification_report

(train_images,train_labels),(test_images,test_labels)=tf.keras.datasets.mnist.load_data()

#Reshape data
train_images=train_images.reshape((60000,28,28,1))
test_images=test_images.reshape((10000,28,28,1))

#Normalize
train_images,test_images=train_images/255.0 ,test_images/255.0


## Redes utilizadas

- Camadas finais de adensamento com função de ativação 'relu' e de categorização com função de ativação 'softmax'.

### CNN1:
- Neurônios: 32 à 128
- Dilatação: 1 ou 2
- Camadas CNN: 2

### CNN2:
- Neurônios: 32 à 128
- Dilatação: 1 ou 2
- Camadas CNN: 3

### CNN3:
- Neurônios: 32 à 128
- Dilatação: 1
- Camadas CNN: 5

In [ ]:
def cnn(trial):
  (x_train, y_train), (x_test, y_test) = (train_images,train_labels),(test_images,test_labels)
  
  activation = trial.suggest_categorical('activation', ['sigmoid', 'softmax', 'tanh', 'relu','softplus','softsign','selu','elu','exponential'])
  
  n_neurons = trial.suggest_int('n_neurons', 16, 64)
  dilation = trial.suggest_int('dilation', 1, 2)
  loss = trial.suggest_categorical('loss', ['sparse_categorical_crossentropy'])
  optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam','rmsprop','adamax'])
  
  print("Função de ativação: {}\nNúmero de neurônios: {}\nOtimizador: {}".format(activation,n_neurons,optimizer))
  model = keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(n_neurons, (dilation,dilation),activation=activation,input_shape=(28,28,1)))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_neurons*2, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(loss=loss,optimizer=optimizer,metrics=["accuracy"])

  history = model.fit(x_train, y_train, epochs=8, verbose=1)
  y_pred = np.argmax(model.predict(x_test), axis=-1)
  y_true = y_test
  return accuracy_score(y_true, y_pred)

In [ ]:
study_1 = optuna.create_study(direction='maximize')
study_1.optimize(cnn, n_trials=16)

[I 2021-11-05 13:13:11,446] A new study created in memory with name: no-name-0212a617-dc7d-4797-b20d-8c84dd82265d


Função de ativação: selu
Número de neurônios: 32
Otimizador: sgd
Epoch 1/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3251 - accuracy: 0.9087
Epoch 2/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1599 - accuracy: 0.9535
Epoch 3/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1184 - accuracy: 0.9643
Epoch 4/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0939 - accuracy: 0.9720
Epoch 5/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0782 - accuracy: 0.9766
Epoch 6/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0670 - accuracy: 0.9797
Epoch 7/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0577 - accuracy: 0.9825
Epoch 8/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0505 - accuracy: 0.9842


[I 2021-11-05 13:13:49,505] Trial 0 finished with value: 0.9729 and parameters: {'activation': 'selu', 'n_neurons': 32, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'sgd'}. Best is trial 0 with value: 0.9729.


Função de ativação: softplus
Número de neurônios: 24
Otimizador: rmsprop
Epoch 1/8
1875/1875 [==============================] - 6s 3ms/step - loss: 2.4701 - accuracy: 0.1115
Epoch 2/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 3/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 4/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 5/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 6/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 7/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 8/8
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124


[I 2021-11-05 13:14:32,475] Trial 1 finished with value: 0.1135 and parameters: {'activation': 'softplus', 'n_neurons': 24, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.9729.


...

[I 2021-11-05 13:22:39,955] Trial 13 finished with value: 0.9862 and parameters: {'activation': 'relu', 'n_neurons': 62, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam'}. Best is trial 12 with value: 0.9893.


Função de ativação: elu
Número de neurônios: 52
Otimizador: adam
Epoch 1/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1505 - accuracy: 0.9547
Epoch 2/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0621 - accuracy: 0.9810
Epoch 3/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0421 - accuracy: 0.9866
Epoch 4/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0361 - accuracy: 0.9883
Epoch 5/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0280 - accuracy: 0.9911
Epoch 6/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0227 - accuracy: 0.9926
Epoch 7/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0220 - accuracy: 0.9928
Epoch 8/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0185 - accuracy: 0.9941


[I 2021-11-05 13:23:14,891] Trial 14 finished with value: 0.9822 and parameters: {'activation': 'elu', 'n_neurons': 52, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam'}. Best is trial 12 with value: 0.9893.


Função de ativação: tanh
Número de neurônios: 53
Otimizador: adam
Epoch 1/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1474 - accuracy: 0.9549
Epoch 2/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0628 - accuracy: 0.9804
Epoch 3/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0419 - accuracy: 0.9865
Epoch 4/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0340 - accuracy: 0.9890
Epoch 5/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0287 - accuracy: 0.9908
Epoch 6/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0219 - accuracy: 0.9927
Epoch 7/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0205 - accuracy: 0.9934
Epoch 8/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0166 - accuracy: 0.9949


[I 2021-11-05 13:23:50,908] Trial 15 finished with value: 0.9828 and parameters: {'activation': 'tanh', 'n_neurons': 53, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam'}. Best is trial 12 with value: 0.9893.


In [ ]:
def cnn2(trial):
  (x_train, y_train), (x_test, y_test) = (train_images,train_labels),(test_images,test_labels)
  
  activation = trial.suggest_categorical('activation', ['sigmoid', 'softmax', 'tanh', 'relu','softplus','softsign','selu','elu','exponential'])
  
  n_neurons = trial.suggest_int('n_neurons', 16, 64)
  dilation = trial.suggest_int('dilation', 1, 2)
  loss = trial.suggest_categorical('loss', ['sparse_categorical_crossentropy'])
  optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam','rmsprop','adamax'])
  
  print("Função de ativação: {}\nNúmero de neurônios: {}\nOtimizador: {}".format(activation,n_neurons,optimizer))
  model = keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(n_neurons, (dilation,dilation),activation=activation,input_shape=(28,28,1)))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))

  # ----------------- ADICIONAMOS MAIS UMA CAMADA CNN ----------------- #
  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  # ----------------- ADICIONAMOS MAIS UMA CAMADA CNN ----------------- #

  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_neurons*2, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(loss=loss,optimizer=optimizer,metrics=["accuracy"])

  history = model.fit(x_train, y_train, epochs=8, verbose=1)
  y_pred = np.argmax(model.predict(x_test), axis=-1)
  y_true = y_test
  return accuracy_score(y_true, y_pred)

In [ ]:
study_2 = optuna.create_study(direction='maximize')
study_2.optimize(cnn2, n_trials=16)

[I 2021-11-05 13:23:50,941] A new study created in memory with name: no-name-fd4b0ef8-84d3-496c-9361-d91ac0a9eb1f


Função de ativação: softmax
Número de neurônios: 20
Otimizador: rmsprop
Epoch 1/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3023 - accuracy: 0.1119
Epoch 2/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 3/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 4/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 5/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 6/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 7/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 8/8
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3014 - accuracy: 0.1124


[I 2021-11-05 13:24:45,081] Trial 0 finished with value: 0.1135 and parameters: {'activation': 'softmax', 'n_neurons': 20, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.1135.


Função de ativação: selu
Número de neurônios: 36
Otimizador: adamax
Epoch 1/8
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2375 - accuracy: 0.9299
Epoch 2/8
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1321 - accuracy: 0.9595
Epoch 3/8
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0994 - accuracy: 0.9701
Epoch 4/8
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0822 - accuracy: 0.9750
Epoch 5/8
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0676 - accuracy: 0.9788
Epoch 6/8
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0559 - accuracy: 0.9821
Epoch 7/8
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0483 - accuracy: 0.9845
Epoch 8/8
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0416 - accuracy: 0.9865


[I 2021-11-05 13:25:46,116] Trial 1 finished with value: 0.9689 and parameters: {'activation': 'selu', 'n_neurons': 36, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adamax'}. Best is trial 1 with value: 0.9689.


...

[I 2021-11-05 13:36:01,848] Trial 13 finished with value: 0.9813 and parameters: {'activation': 'softsign', 'n_neurons': 55, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam'}. Best is trial 12 with value: 0.9917.


Função de ativação: softsign
Número de neurônios: 31
Otimizador: adamax
Epoch 1/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2554 - accuracy: 0.9264
Epoch 2/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0827 - accuracy: 0.9759
Epoch 3/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0575 - accuracy: 0.9829
Epoch 4/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0450 - accuracy: 0.9864
Epoch 5/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0370 - accuracy: 0.9889
Epoch 6/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0315 - accuracy: 0.9906
Epoch 7/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0274 - accuracy: 0.9917
Epoch 8/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0238 - accuracy: 0.9930


[I 2021-11-05 13:36:43,751] Trial 14 finished with value: 0.99 and parameters: {'activation': 'softsign', 'n_neurons': 31, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adamax'}. Best is trial 12 with value: 0.9917.


Função de ativação: softsign
Número de neurônios: 29
Otimizador: adamax
Epoch 1/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2614 - accuracy: 0.9236
Epoch 2/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0841 - accuracy: 0.9745
Epoch 3/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0599 - accuracy: 0.9824
Epoch 4/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0471 - accuracy: 0.9858
Epoch 5/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0388 - accuracy: 0.9882
Epoch 6/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0328 - accuracy: 0.9902
Epoch 7/8
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0283 - accuracy: 0.9914
Epoch 8/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0246 - accuracy: 0.9924


[I 2021-11-05 13:37:22,545] Trial 15 finished with value: 0.9886 and parameters: {'activation': 'softsign', 'n_neurons': 29, 'dilation': 2, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adamax'}. Best is trial 12 with value: 0.9917.


In [ ]:
def cnn3(trial):
  (x_train, y_train), (x_test, y_test) = (train_images,train_labels),(test_images,test_labels)
  
  activation = trial.suggest_categorical('activation', ['sigmoid', 'softmax', 'tanh', 'relu','softplus','softsign','selu','elu','exponential'])
  
  n_neurons = trial.suggest_int('n_neurons', 16, 64)
  # Redução de dilation
  dilation = trial.suggest_int('dilation', 1, 1)
  loss = trial.suggest_categorical('loss', ['sparse_categorical_crossentropy'])
  optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam','rmsprop','adamax'])
  
  print("Função de ativação: {}\nNúmero de neurônios: {}\nOtimizador: {}".format(activation,n_neurons,optimizer))
  model = keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(n_neurons, (dilation,dilation),activation=activation,input_shape=(28,28,1)))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  
  # ----------------- ADICIONAMOS MAIS TRÊS CAMADA CNN ----------------- #
  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.MaxPooling2D((dilation,dilation)))
  # ----------------- ADICIONAMOS MAIS TRÊS CAMADA CNN ----------------- #

  model.add(tf.keras.layers.Conv2D(n_neurons*2, (dilation,dilation),activation=activation))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_neurons*2, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(loss=loss,optimizer=optimizer,metrics=["accuracy"])

  history = model.fit(x_train, y_train, epochs=8, verbose=1)
  y_pred = np.argmax(model.predict(x_test), axis=-1)
  y_true = y_test
  return accuracy_score(y_true, y_pred)

In [ ]:
study_3 = optuna.create_study(direction='maximize')
study_3.optimize(cnn3, n_trials=16)

[I 2021-11-05 13:37:22,583] A new study created in memory with name: no-name-bb39e995-7f55-4052-b3f5-188b7096ee06


Função de ativação: sigmoid
Número de neurônios: 43
Otimizador: adamax
Epoch 1/8
1875/1875 [==============================] - 14s 7ms/step - loss: 2.3924 - accuracy: 0.1113
Epoch 2/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 3/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124
Epoch 4/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124
Epoch 5/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124
Epoch 6/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124
Epoch 7/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124
Epoch 8/8
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3012 - accuracy: 0.1124


[I 2021-11-05 13:39:08,714] Trial 0 finished with value: 0.1135 and parameters: {'activation': 'sigmoid', 'n_neurons': 43, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adamax'}. Best is trial 0 with value: 0.1135.


Função de ativação: softsign
Número de neurônios: 32
Otimizador: rmsprop
Epoch 1/8
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2440 - accuracy: 0.9276
Epoch 2/8
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1273 - accuracy: 0.9620
Epoch 3/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1019 - accuracy: 0.9704
Epoch 4/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0877 - accuracy: 0.9744
Epoch 5/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0781 - accuracy: 0.9776
Epoch 6/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0697 - accuracy: 0.9795
Epoch 7/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0646 - accuracy: 0.9809
Epoch 8/8
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0596 - accuracy: 0.9834


[I 2021-11-05 13:40:32,414] Trial 1 finished with value: 0.9725 and parameters: {'activation': 'softsign', 'n_neurons': 32, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'rmsprop'}. Best is trial 1 with value: 0.9725.


...

[I 2021-11-05 14:01:07,859] Trial 13 finished with value: 0.1135 and parameters: {'activation': 'softplus', 'n_neurons': 27, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'sgd'}. Best is trial 9 with value: 0.9758.


Função de ativação: selu
Número de neurônios: 16
Otimizador: sgd
Epoch 1/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2681 - accuracy: 0.9192
Epoch 2/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1500 - accuracy: 0.9547
Epoch 3/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1193 - accuracy: 0.9638
Epoch 4/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1042 - accuracy: 0.9685
Epoch 5/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0915 - accuracy: 0.9719
Epoch 6/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0842 - accuracy: 0.9745
Epoch 7/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0756 - accuracy: 0.9763
Epoch 8/8
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0708 - accuracy: 0.9783


[I 2021-11-05 14:01:55,763] Trial 14 finished with value: 0.9657 and parameters: {'activation': 'selu', 'n_neurons': 16, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'sgd'}. Best is trial 9 with value: 0.9758.


Função de ativação: selu
Número de neurônios: 49
Otimizador: adam
Epoch 1/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2563 - accuracy: 0.9254
Epoch 2/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1484 - accuracy: 0.9554
Epoch 3/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1156 - accuracy: 0.9656
Epoch 4/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0980 - accuracy: 0.9706
Epoch 5/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0896 - accuracy: 0.9720
Epoch 6/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0717 - accuracy: 0.9774
Epoch 7/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0668 - accuracy: 0.9790
Epoch 8/8
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0588 - accuracy: 0.9815


[I 2021-11-05 14:03:41,345] Trial 15 finished with value: 0.9717 and parameters: {'activation': 'selu', 'n_neurons': 49, 'dilation': 1, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam'}. Best is trial 9 with value: 0.9758.


##Gráficos de resultados do processo de otimização

In [ ]:
optuna.visualization.plot_optimization_history(study_1)

In [ ]:
optuna.visualization.plot_optimization_history(study_2)

In [ ]:
optuna.visualization.plot_optimization_history(study_3)

### Resultados do plot de otimização de cada estudo
Como podemos ver acima, nossa primeira rede de CNN obteve melhor desempenho em sua otimização, dado que foi mais consistente (o primeiro trial do CNN2 e CNN3 foi um outlier).

In [ ]:
optuna.visualization.plot_slice(study_1)

In [ ]:
optuna.visualization.plot_slice(study_2)

In [ ]:
optuna.visualization.plot_slice(study_3)

### Resultado da comparação de parâmetros
- Funções de ativação: no geral, todas as funções de otimização apresentaram bons resultados (como visto no primeiro estudo). Nos outros dois estudos é visível que as funções "softmax", "softplus" e "sigmoid" foram as piores.
- Dilatação: no terceiro estudo, a rede permitiu apenas uso da dilatação com 1, portanto, vamos desconsiderá-lo. Os outros dois estudos apresentaram melhores resultados quando a dilatação foi 2.
- A função de perda utilizada foi a mesma em todos.
- O número de neurônios foi um parâmetro bastante inconstante, não aparentou ser um diferencial em nenhum dos estudos.
- O otimizador que mais se destacou foi o "adam", e em segundo o "sgd", os outros demonstraram ser igualmente bons, com um pouco mais de outliers.

In [ ]:
optuna.visualization.plot_param_importances(study_1)

In [ ]:
optuna.visualization.plot_param_importances(study_2)

In [ ]:
optuna.visualization.plot_param_importances(study_3)

ZeroDivisionError: ignored

### Diferenciais

A função de ativação foi alta nos dois primeiros estudos, sendo um diferencial importante, os outros fatores pouco impactaram. No terceiro estudo houve algum problema no optuna que impossibilitou a análise.